In [33]:
using Surrogates
using Plots
using Statistics
using Random
using DataFrames
using Distances
using LinearAlgebra
default()

In [6]:
# Define the objective waterflow function (8 dimensions)
function f(x)
    r_w = x[1]
    r = x[2]
    T_u = x[3]
    H_u = x[4]
    T_l = x[5]
    H_l = x[6]
    L = x[7]
    K_w = x[8]
    log_val = log(r/r_w)
    return (2*pi*T_u*(H_u - H_l))/ ( log_val*(1 + (2*L*T_u/(log_val*r_w^2*K_w)) + T_u/T_l))
end


n = 250
initial_n = 138
d = 8
lb = [0.05,100,63070,990,63.1,700,1120,9855]
ub = [0.15,50000,115600,1110,116,820,1680,12045]
original_x = sample(initial_n,lb,ub,SobolSample())
original_y = f.(x)

138-element Vector{Float64}:
  12.352996327449631
  97.5632621840658
 118.91903415603889
  49.01538617702001
  60.75347846785299
 162.9204658313606
  78.53510337818457
  20.650322971637376
  24.00701502036944
 124.85114681490505
 104.52346582843872
  60.06898773714285
  36.40201748744536
   ⋮
  64.28474320499525
  12.17636603522977
  20.150519150558743
  72.18156159601037
  88.8874861429921
  38.535010548051815
  75.95055384717737
  89.02584090230852
  88.17776894753304
  25.28165478639068
  39.48306726577925
  85.71377104392586

In [3]:
function splitdf(df, pct)
           @assert 0 <= pct <= 1
           ids = collect(axes(df, 1))
           shuffle!(ids)
           sel = ids .<= nrow(df) .* pct
           return DataFrame(view(df, sel, :)), DataFrame(view(df, .!sel, :))
       end

splitdf (generic function with 1 method)

In [24]:
# Implementing diversity
function calculate_variance(x, models)
    predictions = []
        for model in models
            prediction = model(x)
            append!(predictions, prediction)
        end
    return var(predictions, corrected=false)
end

function diversity_metric(prev_x, new_x, models, lambda = 0.5)
    min_dist = Inf
    variance = calculate_variance(new_x, models)
    for point in prev_x
        new_dist = euclidean(point, new_x)
        if new_dist < min_dist
            min_dist = new_dist
        end
    end
    return (1 - lambda) * sqrt(variance) + lambda * min_dist
end

diversity_metric (generic function with 2 methods)

In [43]:
function calculate_error(point, models, actual, mode="MSE")
    target = actual(point)
    println(target)
    errors = []
    for model in models
        prediction = model(point)
        println(prediction)
        append!(errors, abs(target - prediction))
    end
    if mode == "MSE"
        return mean(errors.^2)
    end
    if mode == "max"
        return maximum(errors)
    end
end

calculate_error (generic function with 2 methods)

In [44]:
total_samples = 1000
prev_points = copy(original_x)
y = copy(original_y)
sample_space = sample(total_samples, lb, ub, SobolSample())
test, orig_train = splitdf(DataFrame(sample_space), 0.2)
train = copy(orig_train)
error = Inf
while error > 47
    kriging_surrogate = Kriging(prev_points, y, lb, ub, p=[1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9])
    my_radial_basis = RadialBasis(prev_points, y, lb, ub)
    x = []
    max_score = 0
    max_index = -1
    for j in 1:size(train)[1]
        point = Tuple(collect(train[j, :]))
        score = diversity_metric(prev_points, point, [my_radial_basis, kriging_surrogate], 0)
        if score > max_score
            max_score = score
            max_index = j
        end
    end
    append!(x, Vector([Tuple(collect(train[max_index, :]))]))
    deleteat!(train, max_index)
    prev_points = vcat(prev_points, x)
    y = f.(prev_points)
    errors = []
    for k in 1:size(test)[1]
        point = Tuple(collect(train[k, :]))
        append!(errors, calculate_error(point, [my_radial_basis, kriging_surrogate], f))
    end
    error = mean(errors)
    println(error)
end

21.90001839931228
86.00005547608725
76.45995293955578
97.37273394796121
36.184545296281016
76.45995293955578
33.463873952809294
102.72102008339056
76.45995293955578
31.63677661277999
66.58762711899055
76.45995293955578
159.3413369128874
64.50322642612898
76.45995293955578
84.3003715275262
86.99790190519684
76.45995293955578
29.012865379464007
78.7847247263561
76.45995293955578
24.440880713375538
79.99314288491769
76.45995293955578
96.47143085214874
56.15069102832166
76.45995293955578
194.00964716978868
40.028307924834166
76.45995293955578
41.19797663815929
94.03406426767944
76.45995293955578
180.48507769724392
54.97922233110785
76.45995293955578
69.20914116666934
60.87776039356311
76.45995293955578
26.33618356659675
130.4567175561096
76.45995293955578
113.889696615875
90.65777835565098
76.45995293955578
85.06300859753586
64.91859032866421
76.45995293955578
95.01598970366304
66.1036497124594
76.45995293955578
128.4636639239418
65.48554885401899
76.45995293955578
40.68082617625494
72.719

93.04356908979446
76.45995293955578
41.05300242960939
47.830186957751096
76.45995293955578
86.69688083247252
76.08363209492876
76.45995293955578
216.8816238192237
57.499622368313396
76.45995293955578
55.305695475604224
65.90781658048661
76.45995293955578
37.813630582267564
81.82451915925049
76.45995293955578
137.7713946548741
97.97548813515311
76.45995293955578
96.2810281754182
102.69117543290929
76.45995293955578
21.647369701114126
55.65356623874601
76.45995293955578
20.841680307542955
67.4464498467259
76.45995293955578
96.94016127722203
119.27477475722418
76.45995293955578
71.37403485859441
50.42452198562415
76.45995293955578
2696.0521490223746
21.90001839931228
82.13656328874401
76.02101595931114
97.37273394796121
36.243826266059386
76.02101595931114
33.463873952809294
102.61589288732193
76.02101595931114
31.63677661277999
67.04747451190633
76.02101595931114
159.3413369128874
64.5452631241476
76.02101595931114
84.3003715275262
86.81349550007008
76.02101595931114
29.012865379464007
7

76.02101595931114
120.38911613173183
117.47156921888904
76.02101595931114
110.91080624841321
97.31221809638691
76.02101595931114
22.800192399452172
90.9749702236627
76.02101595931114
22.518084949899
60.61819786885195
76.02101595931114
74.81137774656827
104.23545737639733
76.02101595931114
146.50918417372108
99.44041344638617
76.02101595931114
26.7479099475501
84.57635478895281
76.02101595931114
65.87066463799285
70.2932192097312
76.02101595931114
165.878662383241
92.54558701249925
76.02101595931114
114.09536195714591
93.12408511931699
76.02101595931114
41.05300242960939
47.68151399762746
76.02101595931114
86.69688083247252
76.16971273779882
76.02101595931114
216.8816238192237
57.81237127473764
76.02101595931114
55.305695475604224
65.41077721701981
76.02101595931114
37.813630582267564
81.59434281145468
76.02101595931114
137.7713946548741
97.91499879861885
76.02101595931114
96.2810281754182
103.36387683620285
76.02101595931114
21.647369701114126
55.72300819518432
76.02101595931114
20.841

75.9835521233819
66.65142953409423
55.64368612678413
75.9835521233819
62.12718367157293
83.8938490365008
75.9835521233819
137.00265143949147
67.7951679774277
75.9835521233819
89.41009416579186
68.8862259204596
75.9835521233819
31.469692472193348
115.13648016438765
75.9835521233819
35.28032980298908
59.01509720753222
75.9835521233819
139.30728094382098
61.7215215736403
75.9835521233819
75.08629149879545
71.46111057527196
75.9835521233819
50.77707307152105
44.1806077556048
75.9835521233819
88.63892516034458
113.12976123479393
75.9835521233819
65.64059004085397
90.74751495341582
75.9835521233819
25.93540597180137
41.93635182739831
75.9835521233819
26.556343573990947
76.87416520831675
75.9835521233819
79.4112604025619
97.95414171647934
75.9835521233819
202.42642209152234
74.0800224316755
75.9835521233819
39.45687280323495
85.86605428088569
75.9835521233819
82.09325905369145
75.76014930465121
75.9835521233819
197.28494503898676
53.70147183745007
75.9835521233819
98.70718164274872
74.1110041

76.04269952155181
56.74881122299658
65.09233527750303
76.04269952155181
28.600728266124367
54.83885668243329
76.04269952155181
18.20645808686957
85.87314230287157
76.04269952155181
121.00493842556308
60.485038565779064
76.04269952155181
143.05942671536093
79.34129825396667
76.04269952155181
64.97074380266523
63.45085378648423
76.04269952155181
59.21121120977573
74.74912912325544
76.04269952155181
148.9103654389534
78.8134815912303
76.04269952155181
109.13468549265777
75.14375101933638
76.04269952155181
34.72540570398675
55.566801500855604
76.04269952155181
25.593529254928907
47.8810750766055
76.04269952155181
131.6312200888044
82.82508963543768
76.04269952155181
113.89900350355923
87.1017790975676
76.04269952155181
40.28217336096209
72.76657047894605
76.04269952155181
142.74474867314834
115.31038964696927
76.04269952155181
88.55714584418672
82.25726957109066
76.04269952155181
16.797869021116778
106.85968050995791
76.04269952155181
125.50658185426582
65.26347029532474
76.04269952155181


...

76.12625882303551
62.12718367157293
81.47087370620432
76.12625882303551
137.00265143949147
69.01174406620544
76.12625882303551
89.41009416579186
69.03701497775614
76.12625882303551
31.469692472193348
80.47550118759887
76.12625882303551
35.28032980298908
58.61445321335623
76.12625882303551
139.30728094382098
59.49660773585447
76.12625882303551
75.08629149879545
71.70049088611404
76.12625882303551
50.77707307152105
60.021309601743724
76.12625882303551
88.63892516034458
95.37099997561154
76.12625882303551
65.64059004085397
93.71878375536016
76.12625882303551
25.93540597180137
43.54101787470108
76.12625882303551
26.556343573990947
80.12054025528073
76.12625882303551
79.4112604025619
101.05613246892864
76.12625882303551
202.42642209152234
92.16249113236927
76.12625882303551
39.45687280323495
78.03591090547604
76.12625882303551
82.09325905369145
70.47541027046753
76.12625882303551
197.28494503898676
56.289635102797774
76.12625882303551
98.70718164274872
71.40398494153908
76.12625882303551
86

76.10723044793731
56.71176049831123
85.48688257780952
76.10723044793731
110.94022293815132
80.78513023381717
76.10723044793731
67.46369622222608
84.07219144726423
76.10723044793731
24.225382503214526
75.28644247830351
76.10723044793731
16.38891030508094
94.4233299371111
76.10723044793731
68.4748015114352
83.40953506564944
76.10723044793731
91.1749252331103
82.12993816893749
76.10723044793731
66.65142953409423
56.86776507376835
76.10723044793731
62.12718367157293
81.71578354341455
76.10723044793731
137.00265143949147
69.0126630588166
76.10723044793731
89.41009416579186
68.53125581167828
76.10723044793731
31.469692472193348
80.58598939962064
76.10723044793731
35.28032980298908
58.76139823069548
76.10723044793731
139.30728094382098
59.443808081253394
76.10723044793731
75.08629149879545
71.8310564135457
76.10723044793731
50.77707307152105
59.822677244428974
76.10723044793731
88.63892516034458
95.72540815960696
76.10723044793731
65.64059004085397
93.71441351182654
76.10723044793731
25.93540

183.78754251718865
82.90611956663383
75.93143912333875
18.124661182315187
75.69420455970783
75.93143912333875
41.68366796357645
59.78407316758529
75.93143912333875
69.77291253654231
78.39619418615666
75.93143912333875
122.55655104665651
68.34893150038823
75.93143912333875
43.966071503066004
75.05649269734772
75.93143912333875
66.07297026651028
46.44643519349768
75.93143912333875
137.62058237592737
87.72297508627378
75.93143912333875
31.52190204179429
73.04113525118407
75.93143912333875
168.48631842553525
91.48769167657315
75.93143912333875
51.65127967855661
48.71468442966329
75.93143912333875
56.71176049831123
86.01538353766716
75.93143912333875
110.94022293815132
80.88046412150243
75.93143912333875
67.46369622222608
84.27404809420761
75.93143912333875
24.225382503214526
75.21358323165327
75.93143912333875
16.38891030508094
94.06411982936754
75.93143912333875
68.4748015114352
83.43997726939142
75.93143912333875
91.1749252331103
82.15037161159705
75.93143912333875
66.65142953409423
56.9

76.28387836125822
105.78296117670621
80.84184056045348
76.28387836125822
34.57218446031446
53.19334188116159
76.28387836125822
63.85303854738947
110.49055619148476
76.28387836125822
86.25582212063655
56.23238214793355
76.28387836125822
41.97133003638445
77.62346935595815
76.28387836125822
69.95178891069871
83.01850752996461
76.28387836125822
186.3296667002348
97.0406397126244
76.28387836125822
56.74881122299658
63.93630781277807
76.28387836125822
28.600728266124367
55.19792927819435
76.28387836125822
18.20645808686957
57.18336215802458
76.28387836125822
121.00493842556308
61.64030735489655
76.28387836125822
143.05942671536093
79.11606563800171
76.28387836125822
64.97074380266523
65.03209707454243
76.28387836125822
59.21121120977573
50.23330514056261
76.28387836125822
148.9103654389534
78.38546489576777
76.28387836125822
109.13468549265777
79.84931567043964
76.28387836125822
34.72540570398675
58.160926860242625
76.28387836125822
25.593529254928907
54.32684594427042
76.28387836125822
131

LoadError: InterruptException:

In [37]:
println(size(prev_points))

(154,)


In [38]:
n_test = 1000
x_test = sample(n_test,lb,ub,GoldenSample());
y_true = f.(x_test);
my_rad = RadialBasis(prev_points, y, lb, ub)
my_krig = Kriging(prev_points, y, lb, ub, p=[1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9])
y_rad = my_rad.(x_test)
y_krig = my_krig.(x_test);

In [39]:
mse_rad = norm(y_true - y_rad,2)/n_test
mse_krig = norm(y_true - y_krig,2)/n_test
print("MSE RadialBasis: $mse_rad    ")
print("MSE Kriging: $mse_krig    ")

MSE RadialBasis: 1.5698430282200377    MSE Kriging: 1.4261068005903885    